In [7]:
from utils.randomstructure import generate_random_prob
from utils.polycasebuilder import generate_random_polynomials, get_arg_mapping
from trash.inputresolver import reassigneq
from graph.graphutils import edges_to_Ein_Eout, sources
from engine.torchengine import AnalyticalSet, EliminateAnalysis, Function, EliminateAnalysisMergeResiduals
from engine.torchdata import generate_optim_functions
from scipy.optimize import minimize
import sympy as sp
import numpy as np
import torch
np.set_printoptions(formatter={'float': lambda x: "{:0.2f}".format(x).rstrip('0').rstrip('.')})

In [3]:
n_eqs, n_vars, sparsity, seed1, seed2 = 10, 13, 1.1, 42, 12345 #1.7
eqv, varinc, dout = generate_random_prob(n_eqs, n_vars, seed1, sparsity)
polynomials, var_mapping, edges, tree, output_set = generate_random_polynomials(eqv, dout, n_eqs)
symb_mapping = get_arg_mapping(var_mapping, symbol_map=True)

indices = {idx: torch.tensor([idx]) for idx in symb_mapping.keys()}
sets = {}
for polyidx, poly in polynomials.items():
    outputvar = symb_mapping[edges[1][polyidx][0]]
    inputvars = [symb_mapping[elt] for elt in edges[0][polyidx]]
    analysis_function = sp.simplify(reassigneq(None, poly, outputvar))
    if isinstance(analysis_function, sp.Float):
        tensor = torch.tensor([float(analysis_function)], dtype=torch.float64)
        f = lambda : tensor
    else:
        f = sp.lambdify(inputvars, analysis_function, torch) 
    residual = sp.lambdify(inputvars+[outputvar], poly, torch)   
    tupl = (edges[0][polyidx]+edges[1][polyidx], residual)
    triplet = (edges[0][polyidx], edges[1][polyidx], f)
    sets[polyidx] = AnalyticalSet(triplet, indices, forceresidual=tupl)

In [5]:
inputids = sorted(sources(*edges_to_Ein_Eout(edges)))
inputvars = [var_mapping[elt][0] for elt in inputids]
fobj = sum([(elt-1)**2 for elt in inputvars])
objective = Function((inputids, sp.lambdify(inputvars, fobj, torch)), indices)

In [10]:
# optim_funcs_MDF = EliminateAnalysis([sets[k].analysis for k in sorted(sets.keys())],
#                                 [objective, lambda x: torch.tensor([]), lambda x: torch.tensor([])])
equality_constraints_AAO = EliminateAnalysisMergeResiduals([],[sets[k].residual for k in sorted(sets.keys())])
optim_funcs_AAO = EliminateAnalysis(functions=[objective, equality_constraints_AAO])

In [11]:
optim_indices_MDF = torch.cat([indices[elt] for elt in inputids])
bnds_problem_MDF = [(0,10) for elt in optim_indices_MDF]

optim_indices_AAO = torch.cat([indices[k] for k in sorted(indices.keys())])
bnds_problem_AAO = [(0,10) for elt in optim_indices_AAO]
#set random seed to be reproducible
torch.manual_seed(43)
x0 = torch.rand(n_vars, dtype=torch.float64)

In [13]:
xguess, obj_function, ineq_function, eq_function, dobj, dineq, deq, hobj = generate_optim_functions(optim_funcs_AAO, optim_indices_AAO, x0, inequality_direction='positive-null', objective=0, equalities=1)

In [137]:
minimize(obj_function, xguess, constraints=constraints, method='SLSQP')

 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 962868.6890955853
       x: [-2.521e-01  9.823e+02 ... -1.086e-07  7.929e+00]
     nit: 93
     jac: [-2.508e+00  1.963e+03 ...  0.000e+00  0.000e+00]
    nfev: 1485
    njev: 93

In [14]:
x0.numpy()

array([0.63, 0.37, 0.18, 0.73, 0.98, 0.51, 0.15, 0.3, 0.42, 0.72, 0.86,
       0.59, 0.05])

In [153]:
sets[9].analysis(x0).numpy()-x0.numpy()

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])